In [1]:
%cd ..

/home/haito/kaggle/rsna-str/workdir


In [2]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append(".")
from src.factory import *
from src.utils import *
from sklearn.metrics import log_loss

In [3]:
DATADIR = Path("../input/rsna-str-pulmonary-embolism-detection/")

train = pd.read_csv(DATADIR / "train.csv")

pre = pd.read_csv(DATADIR / "split.csv")
train = train.merge(pre, on="StudyInstanceUID")

portion = pd.read_csv(DATADIR / "study_pos_portion.csv")
train = train.merge(portion, on="StudyInstanceUID")

z_pos_df = pd.read_csv(DATADIR / "sop_to_prefix.csv").rename(columns={'img_prefix': 'z_pos'})
train = train.merge(z_pos_df, on="SOPInstanceUID")


### train = train.query("fold == 0 or fold == 1")  # now I have fold0,1 only

studies = train.StudyInstanceUID.unique()

# agg = t.groupby("StudyInstanceUID")["SOPInstanceUID"].apply(list)
# agg_one = t.groupby("StudyInstanceUID").first()
# t = t.set_index("SOPInstanceUID")

In [4]:
def get_pred(_path):
    res = load_pickle(_path)
    raw_pred = pd.DataFrame({
        "SOPInstanceUID": res["ids"],
        **res["outputs"],
    })
    return raw_pred
    # return raw_pred.set_index("sop")

def calib_p(arr, factor):  # set factor>1 to enhance positive prob
    return arr * factor / (arr * factor + (1-arr))

In [30]:
oof_f0, fold0_calib_f = get_pred("output/035_pe_present___448/valid.fold0-ep1.picle"), 8.555037588568537
oof_f1, fold1_calib_f = get_pred("output/035_pe_present___448___apex___resume/valid.fold1-ep1.pickle"), 5.72045
oof_f2, fold2_calib_f = get_pred("output/035_pe_present___448___apex___resume/fold2_ep1.pt.valid.pickle"), 3.3448360180113497
oof_f3, fold3_calib_f = get_pred("output/035_pe_present___448___apex/fold3_ep1.pt.valid.pickle"), 11.18792798738602,
oof_f4, fold4_calib_f = get_pred("output/035_pe_present___448___apex___resume/fold4_ep1.pt.valid.pickle"), 5.720451550292213

# BAD
if False:  # pick best one which yields weighted-logloss after calib
    oof_f3, fold3_calib_f = get_pred("output/035_pe_present___448___apex/fold3_ep0.pt.valid.pickle"), 6.541753595870311
    oof_f4, fold4_calib_f = get_pred("output/035_pe_present___448___apex/fold4_ep0.pt.valid.pickle"), 3.8250639579850194
    

if True: ### ==== do calib for each fold
    oof_f0["pe_present_on_image"] = calib_p(oof_f0["pe_present_on_image"], fold0_calib_f)
    oof_f1["pe_present_on_image"] = calib_p(oof_f1["pe_present_on_image"], fold1_calib_f)
    oof_f2["pe_present_on_image"] = calib_p(oof_f2["pe_present_on_image"], fold2_calib_f)
    oof_f3["pe_present_on_image"] = calib_p(oof_f3["pe_present_on_image"], fold3_calib_f)
    oof_f4["pe_present_on_image"] = calib_p(oof_f4["pe_present_on_image"], fold4_calib_f)

oof = pd.concat([oof_f0, oof_f1, oof_f2, oof_f3, oof_f4]).rename(columns={'pe_present_on_image': 'pred0'})

train = train.merge(oof[['pred0', 'SOPInstanceUID']], on="SOPInstanceUID")  # add pred

In [6]:
train_copyed = train.copy()
train.columns

Index(['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID',
       'pe_present_on_image', 'negative_exam_for_pe', 'qa_motion',
       'qa_contrast', 'flow_artifact', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1',
       'leftsided_pe', 'chronic_pe', 'true_filling_defect_not_pe',
       'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate',
       'exam_type', 'fold', 'pe_present_portion', 'z_pos', 'pred0'],
      dtype='object')

In [7]:
""" feature engineer """

' feature engineer '

In [8]:
# train = pd.read_pickle('../input/train_with_position.pkl')
train = train.sort_values(['StudyInstanceUID', 'z_pos'])
# train = train.merge(oof[['pred', 'SOPInstanceUID']], on='SOPInstanceUID')

# test = pd.read_pickle('../input/test_with_position.pkl')
# test = test.sort_values(['StudyInstanceUID', 'z_pos'])
# test = test.merge(test_pred[pe_present_portion], on='SOPInstanceUID')

# train = train.merge(oof_non_weight[['pred_non_weight', 'SOPInstanceUID']], on='SOPInstanceUID')
# test = test.merge(test_pred_non_weight[['pred_non_weight', 'SOPInstanceUID']], on='SOPInstanceUID')
# train['pred_mean'] = (train['pred'] + train['pred_non_weight']) / 2
# test['pred_mean'] = (test['pred'] + test['pred_non_weight']) / 2

train_current_z_pos = train.groupby('StudyInstanceUID')['z_pos'].shift(0)
# test_current_z_pos = test.groupby('StudyInstanceUID')['z_pos'].shift(0)
### for i in range(1, 20):
for i in range(1, 10):

    # train[f'pre_mean{i}'] = train.groupby('StudyInstanceUID')['pred_mean'].shift(i)
    # train[f'post_mean{i}'] = train.groupby('StudyInstanceUID')['pred_mean'].shift(-i)
    train[f'pred0_pre{i}'] = train.groupby('StudyInstanceUID')['pred0'].shift(i)
    train[f'pred0_post{i}'] = train.groupby('StudyInstanceUID')['pred0'].shift(-i)
    # train[f'pre_non_weight{i}'] = train.groupby('StudyInstanceUID')['pred_non_weight'].shift(i)
    # train[f'post_non_weight{i}'] = train.groupby('StudyInstanceUID')['pred_non_weight'].shift(-i)
    
    # test[f'pre_mean{i}'] = test.groupby('StudyInstanceUID')['pred_mean'].shift(i)
    # test[f'post_mean{i}'] = test.groupby('StudyInstanceUID')['pred_mean'].shift(-i)
    # test[f'pre{i}'] = test.groupby('StudyInstanceUID')['pred'].shift(i)
    # test[f'post{i}'] = test.groupby('StudyInstanceUID')['pred'].shift(-i)
    # test[f'pre_non_weight{i}'] = test.groupby('StudyInstanceUID')['pred_non_weight'].shift(i)
    # test[f'post_non_weight{i}'] = test.groupby('StudyInstanceUID')['pred_non_weight'].shift(-i)


# for i in [1]:
#     train[f'pre_z_pos_diff{i}'] = train_current_z_pos - train.groupby('StudyInstanceUID')['pred'].shift(i)
#     # test[f'pre_z_pos_diff{i}'] = test_current_z_pos - test.groupby('StudyInstanceUID')['pred'].shift(i)

# NORMALIZED Z POS
z_max = train.groupby('StudyInstanceUID').z_pos.max().rename('z_pos_max')
train = train.merge(z_max, on='StudyInstanceUID')
train['z_pos_norm'] = train['z_pos'] / train['z_pos_max']
train = train.drop('z_pos_max', axis=1)

In [9]:
train.tail()

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,...,pred0_post5,pred0_pre6,pred0_post6,pred0_pre7,pred0_post7,pred0_pre8,pred0_post8,pred0_pre9,pred0_post9,z_pos_norm
1790589,fffda3f22362,39ca5eaafffe,29e855db7f2b,0,1,0,0,0,0,0,...,NaN,0.001988,NaN,0.002619,NaN,0.003691,NaN,0.002995,NaN,0.976190
1790590,fffda3f22362,39ca5eaafffe,f7ca277a66c2,0,1,0,0,0,0,0,...,NaN,0.001816,NaN,0.001988,NaN,0.002619,NaN,0.003691,NaN,0.982143
1790591,fffda3f22362,39ca5eaafffe,59714fd8dd25,0,1,0,0,0,0,0,...,NaN,0.001840,NaN,0.001816,NaN,0.001988,NaN,0.002619,NaN,0.988095
1790592,fffda3f22362,39ca5eaafffe,b33567349fae,0,1,0,0,0,0,0,...,NaN,0.001995,NaN,0.001840,NaN,0.001816,NaN,0.001988,NaN,0.994048
1790593,fffda3f22362,39ca5eaafffe,53d378d07811,0,1,0,0,0,0,0,...,NaN,0.001884,NaN,0.001995,NaN,0.001840,NaN,0.001816,NaN,1.000000


In [10]:
ids = [c for c in list(train) if 'UID' in c]
targets = [
    'negative_exam_for_pe',
    'indeterminate',
    'chronic_pe',
    'acute_and_chronic_pe',
    'central_pe',
    'leftsided_pe',
    'rightsided_pe',
    'rv_lv_ratio_gte_1',
    'rv_lv_ratio_lt_1',
]
other_targets = [c for c in list(train) if 'pe_present_on_image' in c]
### remove_cols = ['fold', 'path', 'weight', 'qa_contrast', 'qa_motion'] + targets + ids + other_targets
### remove_cols = ['fold', 'path', 'weight', 'qa_contrast', 'qa_motion'] + ['exam_type','flow_artifact','pe_present_portion', 'true_filling_defect_not_pe'] + targets + ids + other_targets
remove_cols = ['fold', 'path', 'weight', 'qa_contrast', 'qa_motion'] + ['exam_type','flow_artifact','pe_present_portion', 'true_filling_defect_not_pe'] + targets + ids + other_targets + ['z_pos']

features = sorted(list(set(list(train)) - set(remove_cols)))
print(features)

['pred0', 'pred0_post1', 'pred0_post2', 'pred0_post3', 'pred0_post4', 'pred0_post5', 'pred0_post6', 'pred0_post7', 'pred0_post8', 'pred0_post9', 'pred0_pre1', 'pred0_pre2', 'pred0_pre3', 'pred0_pre4', 'pred0_pre5', 'pred0_pre6', 'pred0_pre7', 'pred0_pre8', 'pred0_pre9', 'z_pos_norm']


In [11]:
features_copyed = features.copy()

In [12]:
def fobj(pred, data):
    true = data.get_label()
    label = 2*true - 1
    weights = data.weights
    response = -label / (1 + np.exp(label * pred))
    abs_response = np.abs(response)
    grad = response
    hess = abs_response * (1 - abs_response)
    return grad*weights, hess*weights

In [13]:
import torch
bce_func = torch.nn.BCELoss(reduction='none')

def feval2(preds, data):
    scores = bce_func(torch.FloatTensor(preds), torch.FloatTensor(data.label))
    scores = scores * torch.FloatTensor(data.weights)
    return 'weighted logloss', torch.mean(scores), False

In [14]:
#### 推測して作成したもの. Yujiに確認する必要がある
import torch
bce_func_logit = torch.nn.BCEWithLogitsLoss(reduction='none')

def feval(preds, data):
    scores = bce_func_logit(torch.FloatTensor(preds), torch.FloatTensor(data.label))
    scores = scores * torch.FloatTensor(data.weights)
    return 'weighted logloss', torch.mean(scores), False

In [15]:
import torch
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [16]:
features = ['pred']
features = features_copyed

In [18]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
import warnings
warnings.simplefilter('ignore')
import pickle

oof_preds_list = []
test_preds_list = []
models_list = []
target = 'pe_present_on_image'

for i in range(1):
    print(f'=================={i}================')
    if i % 4 == 0:
        params = {'boosting_type': 'gbdt',
            'objective': 'binary',
#             'metric': 'None',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.1,
            'feature_fraction': 0.9,
            'max_depth': 15,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'verbose': 100,
            'early_stopping_rounds': 100,
            'verbose': -1,
            } 
    elif i % 4 == 1:
        params = {
            'max_depth': 4,
            'max_leave': int(0.2 * 2 ** 4),
            'reg_lambda': 1,
            'reg_alpha': 1,
            'subsamples': 0.8,
            'colsample_bytree': 0.7,
            'objective': 'binary',
            'min_data_in_leaf': 0,
            'boosting': 'gbdt',
            'metric': 'None',
            'learning_rate': 0.1,
                      }
    elif i % 4 == 2:
        params = {
            'num_leaves': 19, 
            'min_data_in_leaf': 160,
            'min_child_weight': 0.03,
            'bagging_fraction' : 0.7,
            'feature_fraction' : 0.8,
            'learning_rate' : 0.1,
            'max_depth': -1,
            'reg_alpha': 0.02,
            'reg_lambda': 0.12,
            'objective': 'binary',
            'verbose': 100,
            'boost_from_average': False,
            'metric': 'None',
        }  
    else:
        params = {
            'objective': "binary",
            'metric': 'None',
            'boost_from_average': "false",
            'tree_learner': "serial",
            'max_depth': -1,
            'learning_rate': 0.1,
            'num_leaves': 197,
            'feature_fraction': 0.3,
            'bagging_freq': 1,
            'bagging_fraction': 0.7,
            'min_data_in_leaf': 100,
            'bagging_seed': 11,
            'max_bin': 255,
            'verbosity': -1}    
        
    oof_preds = np.zeros(train.shape[0])
    ### test_preds = np.zeros(test.shape[0])
    val_results = {}
    models = []
    params['random_state'] = i
    iter = 100000
#     for n_fold, (trn_idx, val_idx) in enumerate(kf.split(train, train[target])):

    for n_fold in range(5):
    ### for n_fold in range(2):
        print(f'    ==============fold{n_fold}================')
        tr = train.query(f'fold != {n_fold}')
        val = train.query(f'fold == {n_fold}')
        trn_data = lgb.Dataset(tr[features], label=tr[target])
        trn_data.weights = tr.pe_present_portion.values
        val_data = lgb.Dataset(val[features], label=val[target])
        val_data.weights = val.pe_present_portion.values
        
        clf = lgb.train(params, trn_data, num_boost_round=iter, valid_sets=[trn_data, val_data], valid_names=['train', 'val'],
#                         verbose_eval=200, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
                        feval=feval, fobj = fobj, verbose_eval=2000, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results, )
        file = f'lgbs/lgb_seed{i}_fold{n_fold}.pkl'
        pickle.dump(clf, open(file, 'wb'))
#         models.append(clf)
        oof_preds[train.fold==n_fold] = clf.predict(val[features])
        ### test_preds += clf.predict(test[features]) / 5
#     models_list.append(models)
    oof_preds_list.append(oof_preds)
    ### test_preds_list.append(test_preds)

print(f'-------------------------------------------------------------------------roc_auc: {roc_auc_score(train[target], np.mean(oof_preds_list, axis=0))}')
print(f'----------------------------------------------------------roc_auc using raw pred: {roc_auc_score(train[target], train["pred0"])}')
print(f'------------------------------------------------------------------------------AP: {average_precision_score(train[target], np.mean(oof_preds_list, axis=0))}')
print(f'---------------------------------------------------------------AP using raw pred: {average_precision_score(train[target], train["pred0"])}')

lgb_oof = np.mean(oof_preds_list, axis=0)
lgb_preds = np.mean(test_preds_list, axis=0)
train['lgb_preds'] = sigmoid(lgb_oof)
### test['lgb_preds'] = sigmoid(lgb_preds)

==================0================
    ==============fold0================
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[88]	train's weighted logloss: 0.0108274	val's weighted logloss: 0.0115188
    ==============fold1================
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[99]	train's weighted logloss: 0.0106117	val's weighted logloss: 0.0121366
    ==============fold2================
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > n

In [18]:
bce_func = torch.nn.BCELoss(reduction='none')

lgb_losses = bce_func(torch.FloatTensor(sigmoid(lgb_oof)), torch.FloatTensor(train['pe_present_on_image']))

### torch.mean(lgb_losses*train['weight'].values)
torch.mean(lgb_losses*train['pe_present_portion'].values).item()

0.011431804800068808

In [19]:
# no stacking result
lgb_losses = bce_func(torch.FloatTensor(train['pred0']), torch.FloatTensor(train['pe_present_on_image']))
torch.mean(lgb_losses*train['pe_present_portion'].values).item()

0.013218380246001216

In [20]:
raise 

RuntimeError: No active exception to reraise

# PE_REPESNT -> POS_EXAM

In [52]:
DATADIR = Path("../input/rsna-str-pulmonary-embolism-detection/")

train = pd.read_csv(DATADIR / "train.csv")

pre = pd.read_csv(DATADIR / "split.csv")
train = train.merge(pre, on="StudyInstanceUID")

portion = pd.read_csv(DATADIR / "study_pos_portion.csv")
train = train.merge(portion, on="StudyInstanceUID")

z_pos_df = pd.read_csv(DATADIR / "sop_to_prefix.csv").rename(columns={'img_prefix': 'z_pos'})
train = train.merge(z_pos_df, on="SOPInstanceUID")

studies = train.StudyInstanceUID.unique()

In [53]:
oof = pd.concat([oof_f0, oof_f1, oof_f2, oof_f3, oof_f4]).rename(columns={'pe_present_on_image': 'pred'})

train = train.merge(oof[['pred', 'SOPInstanceUID']], on="SOPInstanceUID")  # add pred

In [80]:
from functools import partial
def grouping(df):
    grouped = pd.DataFrame(df.groupby('StudyInstanceUID')['pred'].mean())
    grouped = grouped.rename(columns={'pred': 'mean'})
    count = df.groupby('StudyInstanceUID')['pred'].count()
    grouped['count_total'] = count

    for i in range(1,10):
        count = df[df.pred>i/10].groupby('StudyInstanceUID')['pred'].count()
        grouped[f'count_over{i/10}'] = count
        grouped[f'count_over{i/10}_ratio'] = count / grouped['count_total']

    for q in [30, 50, 70, 80, 90, 95, 99]:
    # for q in [95]:
        grouped[f'percentile{q}'] = df.groupby('StudyInstanceUID')['pred'].apply(lambda arr: np.percentile(arr, q))

    ma = pd.DataFrame(df.groupby('StudyInstanceUID')['pred'].max())
    grouped['max'] = ma.pred

    grouped = grouped.reset_index().fillna(0)
    return grouped
train_grouped = grouping(train)

train_grouped['fold'] = train.groupby('StudyInstanceUID')['fold'].first().values
train_grouped['negative_exam_for_pe'] = train.groupby('StudyInstanceUID')['negative_exam_for_pe'].first().values
train_grouped['positive_exam_for_pe'] = (1 - train.groupby('StudyInstanceUID')['negative_exam_for_pe'].first().values) * (1 - train.groupby('StudyInstanceUID')['indeterminate'].first().values)

# test_grouped = grouping(test)

In [85]:
# features = list(set(list(train_grouped)) - set(['StudyInstanceUID', 'negative_exam_for_pe']))
# target = 'negative_exam_for_pe'

### features = list(set(list(train_grouped)) - set(['StudyInstanceUID', 'positive_exam_for_pe', 'negative_exam_for_pe', 'fold']))
features = list(set(list(train_grouped)) - set(['StudyInstanceUID', 'positive_exam_for_pe', 'negative_exam_for_pe', 'fold']) - set(['count_total']))
features = sorted(features)
target = 'positive_exam_for_pe'
# target = 'negative_exam_for_pe'

In [86]:
print(features)

['count_over0.1', 'count_over0.1_ratio', 'count_over0.2', 'count_over0.2_ratio', 'count_over0.3', 'count_over0.3_ratio', 'count_over0.4', 'count_over0.4_ratio', 'count_over0.5', 'count_over0.5_ratio', 'count_over0.6', 'count_over0.6_ratio', 'count_over0.7', 'count_over0.7_ratio', 'count_over0.8', 'count_over0.8_ratio', 'count_over0.9', 'count_over0.9_ratio', 'max', 'mean', 'percentile30', 'percentile50', 'percentile70', 'percentile80', 'percentile90', 'percentile95', 'percentile99']


In [87]:
train_grouped.head()

,StudyInstanceUID,mean,count_total,count_over0.1,count_over0.1_ratio,count_over0.2,count_over0.2_ratio,count_over0.3,count_over0.3_ratio,count_over0.4,...,percentile50,percentile70,percentile80,percentile90,percentile95,percentile99,max,fold,negative_exam_for_pe,positive_exam_for_pe
0,0003b3d648eb,0.271476,223,129.0,0.578475,96.0,0.430493,78.0,0.349776,67.0,...,0.156067,0.398198,0.580246,0.817519,0.889579,0.930089,0.961301,0,1,0
1,000f7f114264,0.079362,239,67.0,0.280335,43.0,0.179916,17.0,0.071130,7.0,...,0.002301,0.083110,0.175883,0.266065,0.331230,0.578595,0.781551,3,1,0
2,00102474a2db,0.067501,326,64.0,0.196319,25.0,0.076687,20.0,0.061350,13.0,...,0.010709,0.052737,0.093520,0.170775,0.340288,0.717843,0.838232,3,1,0
3,0038fd5f09f5,0.037987,230,18.0,0.078261,10.0,0.043478,8.0,0.034783,6.0,...,0.002431,0.014937,0.027755,0.081765,0.162589,0.655598,0.956098,4,1,0
4,0045f113e031,0.132310,257,95.0,0.369650,62.0,0.241245,42.0,0.163424,32.0,...,0.041298,0.138919,0.253426,0.458115,0.581166,0.705473,0.757177,4,1,0


In [90]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter('ignore')
import pickle
from sklearn.model_selection import KFold, StratifiedKFold

oof_preds_list = []
test_preds_list = []
models_list = []

for i in range(1):
    print(f'=================={i}================')
    if i % 4 == 0:
        params = {'boosting_type': 'gbdt',
            'objective': 'binary',
#             'metric': 'None',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.1,
            'feature_fraction': 0.9,
            'max_depth': 15,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'verbose': 100,
            'early_stopping_rounds': 100,
            'verbose': -1,
            } 
    elif i % 4 == 1:
        params = {
            'max_depth': 4,
            'max_leave': int(0.2 * 2 ** 4),
            'reg_lambda': 1,
            'reg_alpha': 1,
            'subsamples': 0.8,
            'colsample_bytree': 0.7,
            'objective': 'binary',
            'min_data_in_leaf': 0,
            'boosting': 'gbdt',
#             'metric': 'None',
            'learning_rate': 0.1,
                      }
    elif i % 4 == 2:
        params = {
            'num_leaves': 19, 
            'min_data_in_leaf': 160,
            'min_child_weight': 0.03,
            'bagging_fraction' : 0.7,
            'feature_fraction' : 0.8,
            'learning_rate' : 0.1,
            'max_depth': -1,
            'reg_alpha': 0.02,
            'reg_lambda': 0.12,
            'objective': 'binary',
            'verbose': 100,
            'boost_from_average': False,
#             'metric': 'None',
        }  
    else:
        params = {
            'objective': "binary",
#             'metric': 'None',
            'boost_from_average': "false",
            'tree_learner': "serial",
            'max_depth': -1,
            'learning_rate': 0.1,
            'num_leaves': 197,
            'feature_fraction': 0.3,
            'bagging_freq': 1,
            'bagging_fraction': 0.7,
            'min_data_in_leaf': 100,
            'bagging_seed': 11,
            'max_bin': 255,
            'verbosity': -1}    
        
    oof_preds = np.zeros(train_grouped.shape[0])

    ### test_preds = np.zeros(test_grouped.shape[0])
    val_results = {}
    models = []
    params['random_state'] = i
    iter = 100000
    kf = KFold(n_splits=5, shuffle=True, random_state=72)

    #for n_fold, (trn_idx, val_idx) in enumerate(kf.split(train_grouped, train_grouped[target])):
        # tr = train_grouped.iloc[trn_idx]
        # val = train_grouped.iloc[val_idx]
    for n_fold in range( 5 ):
        tr = train_grouped[train_grouped.fold != n_fold]
        val = train_grouped[train_grouped.fold == n_fold]
        trn_data = lgb.Dataset(tr[features], label=tr[target])
        val_data = lgb.Dataset(val[features], label=val[target])
        
        clf = lgb.train(params, trn_data, num_boost_round=iter, valid_sets=[trn_data, val_data], valid_names=['train', 'val'],
#                         feval=feval, verbose_eval=10, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
                        verbose_eval=200, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
        file = f'lgbs/lgb_seed{i}_fold{n_fold}.pkl'
        pickle.dump(clf, open(file, 'wb'))
        models.append(clf)
        
        oof_preds[train_grouped.fold == n_fold] = clf.predict(val[features])
        ### oof_preds[val_idx] = clf.predict(val[features])
        ### test_preds += clf.predict(test_grouped[features]) / 5
#     models_list.append(models)
    oof_preds_list.append(oof_preds)
    ###test_preds_list.append(test_preds)

print(f'--------------------------------------------------------------------roc: {roc_auc_score(train_grouped[target], np.mean(oof_preds_list, axis=0))}')
print(f'----------------------------------------------------------------logloss: {log_loss(train_grouped[target], np.mean(oof_preds_list, axis=0))}')
lgb_oof_exam = np.mean(oof_preds_list, axis=0)
### lgb_preds = np.mean(test_preds_list, axis=0)

==================0================
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[41]	train's binary_logloss: 0.25003	val's binary_logloss: 0.316147
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[45]	train's binary_logloss: 0.24036	val's binary_logloss: 0.328105
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[30]	train's binary_logloss: 0.266725	val's binary_logloss: 0.334468
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 round

In [89]:
bce_func = torch.nn.BCELoss(reduction='mean')
lgb_losses = bce_func(torch.FloatTensor(oof_preds), torch.FloatTensor(train_grouped['positive_exam_for_pe']))

torch.mean(lgb_losses).item()

0.33421462774276733

In [71]:
bce_func = torch.nn.BCELoss(reduction='mean')
lgb_losses = bce_func(
    ( 1 - torch.FloatTensor(oof_preds) ) * (4911) / (4911 + 157), 
    torch.FloatTensor(train_grouped['negative_exam_for_pe']))

torch.mean(lgb_losses).item()

0.3707713186740875

In [29]:
# # current yama's pipeline for fold0-ep1
# def calib_p(arr, factor):  # set factor>1 to enhance positive prob
#     return arr * factor / (arr * factor + (1-arr))
# def post_yama(arr):
#     return calib_p( np.percentile(arr, 95), factor=1/8.5550)

# lgb_losses = bce_func(torch.FloatTensor(train[['StudyInstanceUID','pred']].groupby('StudyInstanceUID').apply(post_yama)), torch.FloatTensor(train_grouped['positive_exam_for_pe'])).item()
# lgb_losses